In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS = 50
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                  train=True,
                  download=True,
                  transform = transforms.Compose([
                      transforms.RandomHorizontalFlip(),
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081))
                  ])),
    batch_size = BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                  train=False,
                  transform = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,), (0.3081))
                  ])),
    batch_size = BATCH_SIZE, shuffle=True)

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw


Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw


Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw


Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw
Processing...
Done!


/home/herbwood/anaconda3/envs/myPytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629401553/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training,
                     p=self.dropout_p)
        x = self.fc3(x)
        
        return x

In [6]:
model = Net(dropout_p=0.2).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss : 0.5310, Accuracy : 83.20%
[2] Test Loss : 0.4084, Accuracy : 87.35%
[3] Test Loss : 0.3476, Accuracy : 89.27%
[4] Test Loss : 0.2866, Accuracy : 91.23%
[5] Test Loss : 0.2508, Accuracy : 92.35%
[6] Test Loss : 0.2238, Accuracy : 93.13%
[7] Test Loss : 0.2026, Accuracy : 93.78%
[8] Test Loss : 0.1864, Accuracy : 94.37%
[9] Test Loss : 0.1734, Accuracy : 94.82%
[10] Test Loss : 0.1631, Accuracy : 95.08%
[11] Test Loss : 0.1584, Accuracy : 94.91%
[12] Test Loss : 0.1467, Accuracy : 95.49%
[13] Test Loss : 0.1472, Accuracy : 95.32%
[14] Test Loss : 0.1396, Accuracy : 95.64%
[15] Test Loss : 0.1367, Accuracy : 95.65%
[16] Test Loss : 0.1301, Accuracy : 95.95%
[17] Test Loss : 0.1278, Accuracy : 95.96%
[18] Test Loss : 0.1224, Accuracy : 96.12%
[19] Test Loss : 0.1196, Accuracy : 96.17%
[20] Test Loss : 0.1163, Accuracy : 96.17%
[21] Test Loss : 0.1167, Accuracy : 96.26%
[22] Test Loss : 0.1156, Accuracy : 96.15%
[23] Test Loss : 0.1099, Accuracy : 96.46%
[24] Test Loss : 0.1